In [3]:
import pandas as pd
import seaborn as sns

# Load the Iris dataset
iris = sns.load_dataset('iris')

petal_length = iris['petal_length']
petal_width = iris['petal_width'] 
sepal_length = iris['sepal_length']
sepal_width = iris['sepal_width']


from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
## Eski
"""
import torch
import torchvision

# Load the CIFAR-10 dataset
cifar10_dataset = torchvision.datasets.CIFAR10(root='path/to/data', 
                                               train=True, 
                                               download=True)
data, labels = cifar10_dataset[0]

# Print the data and labels
print(data)
print(labels)

class_names = torchvision.datasets.CIFAR10.classes
print(class_names)

label_name = class_names[labels]
print(label_name)
"""

# Load the raw CIFAR-10 data.
cifar10_dir = 'comp411/datasets/cifar-10-batches-py'

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)





ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [ ]:
# Split the data into train, val, and test sets. In addition we will
# create a small development set as a subset of the training data;
# we can use this for development so our code runs faster.
num_training = 49000
num_validation = 1000
num_test = 1000
num_dev = 500

# Our validation set will be num_validation points from the original
# training set.
mask = range(num_training, num_training + num_validation)
X_val = X_train[mask]
y_val = y_train[mask]

# Our training set will be the first num_train points from the original
# training set.
mask = range(num_training)
X_train = X_train[mask]
y_train = y_train[mask]

# We will also make a development set, which is a small subset of
# the training set.
mask = np.random.choice(num_training, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

# We use the first num_test points of the original test set as our
# test set.
mask = range(num_test)
X_test = X_test[mask]
y_test = y_test[mask]

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
# Preprocessing: reshape the image data into rows
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_val = np.reshape(X_val, (X_val.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

# As a sanity check, print out the shapes of the data
print('Training data shape: ', X_train.shape)
print('Validation data shape: ', X_val.shape)
print('Test data shape: ', X_test.shape)
print('dev data shape: ', X_dev.shape)

In [ ]:


# Preprocessing: subtract the mean image
# first: compute the image mean based on the training data
mean_image = np.mean(X_train, axis=0)
print(mean_image[:10]) # print a few of the elements
plt.figure(figsize=(4,4))
plt.imshow(mean_image.reshape((32,32,3)).astype('uint8')) # visualize the mean image
plt.show()

# second: subtract the mean image from train and test data
X_train -= mean_image
X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

# third: append the bias dimension of ones (i.e. bias trick) so that our SVM
# only has to worry about optimizing a single weight matrix W.
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)

In [4]:
def recall(y_true, y_pred):
    """
    Compute the recall metric for a classification model.
    
    Parameters:
    - y_true: ndarray, shape (n_samples,)
        True labels.
    - y_pred: ndarray, shape (n_samples,)
        Predicted labels.
        
    Returns:
    - recall: float
        Recall metric.
    """
    # Count true positives
    tp = sum((y_true == 1) & (y_pred == 1))
    # Count false negatives
    fn = sum((y_true == 1) & (y_pred == 0))
    # Compute recall
    recall = tp / (tp + fn)
    return recall

In [5]:
def f1_score(y_true, y_pred):
    """
    Compute the F1 score metric for a classification model.
    
    Parameters:
    - y_true: ndarray, shape (n_samples,)
        True labels.
    - y_pred: ndarray, shape (n_samples,)
        Predicted labels.
        
    Returns:
    - f1_score: float
        F1 score metric.
    """
    # Count true positives
    tp = sum((y_true == 1) & (y_pred == 1))
    # Count false positives
    fp = sum((y_true == 0) & (y_pred == 1))
    # Count false negatives
    fn = sum((y_true == 1) & (y_pred == 0))
    # Compute precision and recall
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    # Compute F1 score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [6]:
def accuracy(y_true, y_pred):
    """
    Compute the accuracy metric for a classification model.
    
    Parameters:
    - y_true: ndarray, shape (n_samples,)
        True labels.
    - y_pred: ndarray, shape (n_samples,)
        Predicted labels.
        
    Returns:
    - accuracy: float
        Accuracy metric.
    """
    # Count true predictions
    n_correct = sum(y_true == y_pred)
    # Compute accuracy
    accuracy = n_correct / len(y_true)

In [7]:
def precision(y_true, y_pred):
    """
    Compute the precision metric for a classification model.
    
    Parameters:
    - y_true: ndarray, shape (n_samples,)
        True labels.
    - y_pred: ndarray, shape (n_samples,)
        Predicted labels.
        
    Returns:
    - precision: float
        Precision metric.
    """
    # Count true positives
    tp = sum((y_true == 1) & (y_pred == 1))
    # Count false positives
    fp = sum((y_true == 0) & (y_pred == 1))
    # Compute precision
    precision = tp / (tp + fp)
    return precision

In [5]:
petal_width

0      0.2
1      0.2
2      0.2
3      0.2
4      0.2
      ... 
145    2.3
146    1.9
147    2.0
148    2.3
149    1.8
Name: petal_width, Length: 150, dtype: float64